In [1]:
import lsdtopytools as lsd, glob
import os

/sciclone/home/ntlewis/Nick-Lewis-Research/.conda/lib/python3.8/site-packages/lsdtopytools/geoconvtools.py:79: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit(nopython = False)
/sciclone/home/ntlewis/Nick-Lewis-Research/.conda/lib/python3.8/site-packages/lsdtopytools/numba_tools.py:73: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit(nopython=False,

In [2]:
# Every time you change something within the code cell, run it once more to update the variable.

BOUNDS = [38.985453, -78.975131, 38.336868, -80.04401] # In decimal coordinate form.  Format: [N, E, S, W].  Select the smallest area possible in your analyses for maximum resoultion!  

API_KEY = '9086887f175d33aa72eda767f5b1e9cd' # Get your API key from opentopography.org.  Students get higher resolution data!

DEM_DATASET = 'USGS30m' # Options are USGS30m, USGS10m, & USGS1m.  Be warned!  As resolution goes up, available download area goes down.

PROJ_TITLE = 'wv_lower' # Create a succinct name with no spaces or leading digits to represent your project file for future exports.

PATH = os.getcwd()

In [3]:
names = glob.glob(f'{PATH}/{PROJ_TITLE}/reproj/*.tiff')
names = [path.split(sep='/')[-1] for path in names]
names

['Middle_Jackson_River_reproj.tiff',
 'Upper_Jackson_River_reproj.tiff',
 'North_Fork_South_Branch_Potomac_River_reproj.tiff',
 'Deer_Creek-Greenbrier_River_reproj.tiff',
 'Upper_Elk_River_reproj.tiff',
 'Lost_River_reproj.tiff',
 'Shavers_Fork_reproj.tiff',
 'Upper_South_Branch_Potomac_River_reproj.tiff',
 'Upper_Cowpasture_River_reproj.tiff',
 'South_Fork_South_Branch_Potomac_River_reproj.tiff',
 'Dry_River_reproj.tiff',
 'Lower_North_River_reproj.tiff',
 'Lower_South_Branch_Potomac_River_reproj.tiff',
 'Sitlington_Creek-Greenbrier_River_reproj.tiff',
 'Glady_Fork_reproj.tiff',
 'Calfpasture_River_reproj.tiff',
 'Dry_Fork_reproj.tiff',
 'Upper_Tygart_Valley_River_reproj.tiff',
 'Upper_North_River_reproj.tiff',
 'Middle_Tygart_Valley_River_reproj.tiff',
 'Middle_Fork_River_reproj.tiff',
 'Shoemaker_River-North_Fork_Shenandoah_River_reproj.tiff',
 'South_Mill_Creek-Mill_Creek_reproj.tiff']

In [4]:
def make_ksn_df(dem_name, path=f'{PATH}/{PROJ_TITLE}/reproj/'):
    name = dem_name[:-5]
    mydem = lsd.LSDDEM(path, dem_name)
    print('Raster Loaded')
    mydem.PreProcessing()
    print('PreProcessed')
    mydem.CommonFlowRoutines()
    print('Flow routines got')
    mydem.ExtractRiverNetwork()
    print('River Network Extracted')
    mydem.DefineCatchment(method='main_basin')
    print('Catchment defined')
    mydem.GenerateChi()
    print('Chi generated')
    mydem.ksn_MuddEtAl2014(target_nodes=70, n_iterations=60, skip=1, minimum_segment_length=10, sigma=2,  nthreads = 1, reload_if_same = False)
    mydem.knickpoint_extraction(lambda_TVD = "auto", combining_window = 30, window_stepped = 80, n_std_dev = 7)
    knickpoint_df = mydem.df_knickpoint
    knickpoint_df.to_csv(f'{PATH}/{PROJ_TITLE}/ksn_csvs/{name}_knickpoints.csv')

In [5]:
try:
    os.mkdir(f'{PATH}/{PROJ_TITLE}/ksn_csvs')
except FileExistsError:
    print('Directory exists')
for name in names:
    make_ksn_df(name)

Loading the raster from file: /sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/wv_lower/reproj/Middle_Jackson_River_reproj.tiff
LOADING TOOK 0.24384665489196777
I am recasting your nodata values to -9999 (standard LSDTT)
PREPROC TOOK 0.0020585060119628906
Alright, let me summon control upon the c++ code ...
I am an empty LSDJunctionNetwork
Got it.
INGESTINGINTO CPP TOOK 0.00636744499206543
Nrows:2277 and ncols:1414
out Nrows:2277 and out ncols:1414

I am an empty LSDJunctionNetwork
Yes, yes, we know. Ignore the above depressing debugging text, that's perfectly normal to feel empty sometimes.
FINALISATION TOOK 0.44908618927001953
lsdtopytools is now ready to roll!
Raster Loaded
Carving: implementation of Lindsay (2016) DOI: 10.1002/hyp.10648
Filling: implementation of Wang and Liu (2006): https://doi.org/10.1080/13658810500433453
Processing...
I am going to carve/breach your depressions in order to force flow paths.
I am using an algorithm from Lindsay et al., 2016